In [1]:
import pandas as pd
url = 'https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/10.%20RNN%20Text%20Classification/dataset/spam.csv'

In [2]:
df = pd.read_csv(url, encoding='latin1')
df.head(3)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN


In [3]:
df = df[['v1','v2']]
df.head(3)

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...


In [4]:
df.isna().sum().sum()

0

In [5]:
df.shape, df.v2.nunique()

((5572, 2), 5169)

In [6]:
df.drop_duplicates(subset=['v2'], inplace=True)
df.shape

(5169, 2)

In [7]:
df.v2 = df.v2.str.replace('[^A-Za-z]',' ',regex=True)

In [63]:
df.v1 = df.v1.replace(['ham','spam'],[0,1])
df.head(3)

,v1,v2
0,0,Go until jurong point crazy Available only ...
1,0,Ok lar Joking wif u oni
2,1,Free entry in a wkly comp to win FA Cup fina...


In [64]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df.v2.values, df.v1.values, stratify=df.v1.values, test_size=0.2, random_state=2023
)

# 2

In [65]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [66]:
tvect = TfidfVectorizer(ngram_range=(1,2), stop_words='english')
lrc = LogisticRegression(random_state=2023)
pipeline = Pipeline([('TVECT',tvect),('LRC',lrc)])
%time pipeline.fit(X_train, y_train)

CPU times: total: 312 ms
Wall time: 329 ms


Pipeline(steps=[('TVECT',
                 TfidfVectorizer(ngram_range=(1, 2), stop_words='english')),
                ('LRC', LogisticRegression(random_state=2023))])

In [67]:
pipeline.score(X_test, y_test)

0.9410058027079303

In [80]:
from sklearn.model_selection import GridSearchCV
params = {'TVECT__max_df': [100, 200],'LRC__C': [20,30]}

In [81]:
grid_pipe = GridSearchCV(pipeline, params, scoring='accuracy', cv=3)
%time grid_pipe.fit(X_train, y_train)

CPU times: total: 5.2 s
Wall time: 5.34 s


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('TVECT',
                                        TfidfVectorizer(ngram_range=(1, 2),
                                                        stop_words='english')),
                                       ('LRC',
                                        LogisticRegression(random_state=2023))]),
             param_grid={'LRC__C': [20, 30], 'TVECT__max_df': [100, 200]},
             scoring='accuracy')

In [82]:
grid_pipe.best_params_

{'LRC__C': 30, 'TVECT__max_df': 200}

In [83]:
best_pipe = grid_pipe.best_estimator_
best_pipe.score(X_test, y_test)

0.9709864603481625